본 예제는 pseudo code를 표현하였습니다.  
동형암호 연산에서 반드시 필요한 **rescale**, **moddown** 작업을 보여드립니다. 

암호문의 기본 특성은
1. slot 개수 (고정)
2. scale (가변)
3. noise budget (가변) 
이 있습니다. 

연산에 사용되는 두 암호문은 scale과 mod index (혹은 level, noise budget라고도 표현)가 맞아야합니다.  

In [5]:
# 기존 함수
def fun(x,y):
    return (x**2+y**2)**3

# 동형암호 버전
def fun_fhe_ver1(x,y):
    """pseudo code.
       *, +은 FHE 전용 연산을 표현
       x: scale = 30, level = 0
       y: scale = 30, level = 0
       1. 예제에서 input 변수의 'scale'은 기본값 30 (2^30)으로 가정
       2. 예제에서 input 변수의 mod(level)는 0으로 가정 
    """
    x2 = x*x 
    # 곱셈 후 x2.scale = x.scale + x.scale = 60으로 증가.
    rescale(x2) # 60 -> 30으로 scale down하면 대신 곱셈 깊이(level) 증가 (scale:30, level:1)
    y2 = y*y # (scale:60, level:0)
    rescale(y2) # 60 -> 30으로 scale down => (s:30, l:1)
    z = x2 + y2 # (s:30, l:1) 덧셈은 level 증가, scale 증가 없음, 
    z2 = z * z # (s:60, l:1)
    rescale(z2) # (s:30, l:2)
    
    moddown(z) # z2와 계산 가능하도록 임의로 level 상향. l1=1 -> l=2, (s:30, l:2)
    z3 = z2*z # == z * (z * z) => (s:60, l:2)
    return z3 # scale = 60인 암호문을 반환 

1. Binary Op에서 암호문의 scale이 동일하고, 곱셈 깊이가 동일해야 연산 가능합니다
2. 모든 연산은 Unary 혹은 Binary

In [14]:
def fun_fhe_ver2(x,y):
    x2 = x*x # (s:60, l=0)
    y2 = y*y # # (s:60, l=0)
    # !!! x2, y2 rescale skipped !!!
    z = x2 + y2 # (s:60, l:0)
    rescale(z) # (s:30, l:1)
    z2 = z*z # (s:60, l:1)
    rescale(z2) # (s:30, l:2)   
    moddown(z) # (s:30, l:2)
    
    z3 = z2*z # == z * (z * z) => (s:60, l:2)
    return z3 

- ver1에서는 rescale이 3회, ver2에서는 rescale이 2회 쓰였습니다
- 최적의 rescale, moddown 지점 (그리고 bootstrap 지점)을 컴파일러 최적화로 결정하려합니다 
- 이 예제에서 연산 cost는 operand의 `scale`, `level`, 그리고 `built-in function`들의 cost의 함수입니다
- f_cost = f_cost(s1, s2, l1, l2, c_rescale, c_moddown, c_mult, c_add)
- 곱셈 연산은 operand의 `level`에 따라 속도가 달라집니다  
- c_mult = c_mult(l1,l2) 

따라서 암호문의 level, scale이 컴파일러의 최적화 pass에서 사용될 예정입니다. 